In [1]:
# %%capture
# # If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed
# !pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
# !pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/accelerate-0.27.2-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
# !pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/transformers-4.38.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
# !pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/optimum-1.17.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install transformers -f /kaggle/input/transformerslib --no-index
!pip install bitsandbytes -f /kaggle/input/transformerslib --no-index

Looking in links: /kaggle/input/transformerslib
Looking in links: /kaggle/input/transformerslib
Processing /kaggle/input/transformerslib/bitsandbytes-0.42.0-py3-none-any.whl


In [2]:
add_special_tokens = False
max_length = 1024
debug = False

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

MODEL_PATH = "/kaggle/input/gemma7bit"
# Found a good blog to catch me up fast!
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
# https://huggingface.co/docs/transformers/v4.38.1/en/quantization#compute-data-type
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
#     torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

# model = model.to_bettertransformer()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
train = pd.read_csv('/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv')
test = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')
sample_sub = pd.read_csv('/kaggle/input/llm-prompt-recovery/sample_submission.csv')

prompts_txt = list(train.rewrite_prompt.unique())
prompts = tokenizer.batch_encode_plus(prompts_txt,add_special_tokens=add_special_tokens,return_attention_mask=False)['input_ids']
n_prompt = len(prompts)
if debug:
    original_texts = train.original_text.tolist()[:5]
    rewritten_text = train.rewritten_text.tolist()[:5]
else:
    original_texts = test.original_text.tolist()
    rewritten_text = test.rewritten_text.tolist()    
original_texts = tokenizer.batch_encode_plus(original_texts,add_special_tokens=add_special_tokens,\
                                             return_attention_mask=False,truncation=True, max_length=max_length)['input_ids']
rewritten_texts = tokenizer.batch_encode_plus(rewritten_text,add_special_tokens=add_special_tokens,\
                                              return_attention_mask=False,truncation=True, max_length=max_length)['input_ids']

In [5]:
def combine_tokens(org, prompt, rewrt, tokenizer):
    # inputs are list of tokens
    a = tokenizer.encode("Original Essay:", add_special_tokens=False)
    b = tokenizer.encode("Prompt:",add_special_tokens=False)
    c = tokenizer.encode("Rewritten Essay:",add_special_tokens=False)
    return a + org + b + prompt + c + rewrt

def create_batch_txt(original_texts, rewritten_texts, prompts):
    # all inputs should be list to token_ids
    out = []
    rewritten_text_len = []
    for org,rewrt in zip(original_texts,rewritten_texts):
        n = len(rewrt)
        for prompt in prompts:
            out.append(torch.tensor(combine_tokens(org, prompt, rewrt,tokenizer),dtype=torch.long))
            tot_len = out[-1].shape[0]
            rewritten_text_len.append((tot_len-n,tot_len)) # start and end of rewritten_texts
    return out,rewritten_text_len

# def gen_batch(token_ids, rewritten_text_len, batch_size):
#     n = len(token_ids)
#     for i in range(min(n//batch_size+1,n)):
#         tokenList,rewt_lenList = token_ids[i*batch_size:i*batch_size+batch_size],rewritten_text_len[i*batch_size:i*batch_size+batch_size]
#         inputs = torch.nn.utils.rnn.pad_sequence(tokenList, batch_first=True, padding_value=tokenizer.pad_token_id).to('cuda')
#         yield inputs,rewt_lenList

In [6]:
token_ids, rewritten_text_len = create_batch_txt(original_texts, rewritten_texts, prompts)

In [7]:
loss = nn.CrossEntropyLoss()
CE_out = []
with torch.no_grad():
    for inputs,rewt_len in zip(token_ids, rewritten_text_len):
        inputs = torch.tensor(inputs,dtype=torch.long)[None].to('cuda')
        outs = model(inputs,output_attentions=False,output_hidden_states=False,use_cache=False).logits
        i,o,(s,e) = inputs[0],outs[0],rewt_len
        # rewt_len is start and end of rewritten_texts
        # shift by one as k-th output logits has access to k-th input
        CE_out.append(loss(o[s:e-1], i[s+1:e]).item())

/tmp/ipykernel_26/3529394721.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.long)[None].to('cuda')
2024-03-02 17:38:57.310833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 17:38:57.310933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 17:38:57.458190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_26/3529394721.py:5: UserWarning: To copy construct fr

In [8]:
CE_out = np.array(CE_out).reshape(test.shape[0],n_prompt)
prompt_idx = CE_out.argmin(-1)

In [9]:
sample_sub["rewrite_prompt"] = [prompts_txt[idx] for idx in prompt_idx]
sample_sub.to_csv("submission.csv", index=False)